In [ ]:
import pickle
import pandas as pd
import flair
from flair.splitter import SegtokSentenceSplitter
from flair.splitter import SegtokTokenizer
from io import BytesIO
import textract
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
import platform
import re
import nltk
import openai
import os
import numpy as np

def read_all_pdf():
    escape_seq = '\n'
    excel, ppt, word = [], [], []
    err_files = []
    err_log = []
    file_names = []
    page_wise_data = []
    folder_path = os.path.join('Documents', 'AIA_Basefiles') #input("Enter folder location: ")

    file_n, file_info = [],[]

    str1=[]

    # Walk through all files and subdirectories in the folder
    for root, dirs, files in os.walk(folder_path):
        try:
            for index, file in enumerate(files):
                if file.endswith(".pdf"):
                    
                    pdf_location = os.path.abspath(os.path.join(root, file))
                    print(pdf_location)
                    page_wise_data = pdfdata(pdf_location)
                    
                    file_n.append(file)
                    file_info.append("\n".join(page_wise_data))

                    #dataframe = df.assign(Contents = pa\ge_wise_data)
                    # df['Contents'] = page_wise_data   #Extract text from PDF
                    file_names.append(file)
                    print(f'[File Number]:: {index}\n [File Name]:: {file_n[-1]}\n[File count:]{len(file_n)}')
                    # count += 1
                elif file.endswith(".xlsx") or file.endswith(".csv") or file.endswith(".xlsb"):
                    excel.append(file)
                elif file.endswith(".ppt"):
                    ppt.append(file)
                elif file.endswith(".docx") or file.endswith(".doc"):
                    word.append(file)
                    print(f'[File Number]:: {index}\n [File Name]:: {file_n[-1]}\n[File count:]{len(file_n)}')
                    doc_location = os.path.abspath(os.path.join(root, file))
                    text = textract.process(doc_location)
                    file_names.append(file)
                    file_info.append(text)
                    
        except Exception as e:
            
            print(f'///////////////////////////\n\n |Random ERORR| {e}, continuing PDF extraction\n\n/////////////////////////////////////')
            err_files.append(pdf_location)
            err_log.append(e)
            continue 
    print(f'Lengths: {len(file_info)} {len(file_n)}')
    return(file_n, file_info)            

def pdfdata(loc):
    escape_seq = '\n'
    rsrcmgr = PDFResourceManager()
    retstr = BytesIO()
    codec = 'utf-8'
    data = []
    laparams = LAParams(char_margin=30, line_margin=2, boxes_flow=1)
    device = TextConverter(rsrcmgr, retstr, codec='utf-8', laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    caching = True
    fp = open(loc, 'rb')
    pagenumber = 0
    pagenos=set()
    for pagenumber, page in enumerate(PDFPage.get_pages(fp,pagenos, maxpages=0,caching=True, check_extractable=True)):
        #Process pages one by one into interpreter
        if pagenumber is not None:
            temp_str = ''
            interpreter.process_page(page)
            if len(retstr.getvalue()) < 25:
               continue
            else:
                temp_str = retstr.getvalue().decode('ascii', 'ignore')  # add extracted text from bytesIO to data variable
                temp_str = temp_str.replace('\x0c \xe2 \x80 \x9d', '')    # Remove useless character
                temp_str = re.sub('\n+', '. ', temp_str)
                data.append(temp_str)
        retstr.truncate(0)
        retstr.seek(0)
                
    # print(f' >>>>>>NORMALSTART<<<<<< {escape_seq.join(data)} >>>>>>>>END<<<<<<<')
    #print(f' >>>>>>START<<<<<< {sent_text} >>>>>>>>END<<<<<<<')
    return(data)
    
def create_dict(folder):
    file_n, file_info = read_all_pdf()
    file_path = os.path.join(folder, 'raw_files.pkl')
    # dictt = {"File Name": file_n, "File Text": file_info}
    # n=1
    cleaned_text=[]  
    matched_names = []                                       
    for i in range(0,len(file_n)-1):
        print(f'File under check: {file_n[i]}')
        file_raw_text = str(file_info[i])
        file_raw_text = file_raw_text.replace("\n",". ")
        cleaned_text.append(file_raw_text)
        matched_names.append(file_n[i])
        file_raw_text=[]
        
    print(f'Printing length:: {len(matched_names)},{len(cleaned_text)}')


    final_dic ={"title": matched_names , "text": cleaned_text}
    df = pd.DataFrame.from_dict(final_dic)
    
    df.to_pickle(file_path)
    chuncked_files(df, folder)

def chuncked_files(df, folder):
    sentence_splitter = SegtokSentenceSplitter()
    word_tokeniser = SegtokTokenizer()
    chunk = []
    token_counter = 0
    dic_text = []
    dic_title = []
    limit = 2000
    
    location = os.path.join(folder, 'GPT_Chuncked.pkl')

    new_df = pd.DataFrame(columns = ['title', 'text'])


    for i in range(len(df)):
        
        text = df.loc[i, 'text']
        title = df.loc[i, 'title']
        # print(f'Text in process: {text}')
        sentences = sentence_splitter.split(text)
        n_tokens = [len(word_tokeniser.tokenize(sentence.text)) for sentence in sentences]
        
        for sentence, tokens in zip(sentences, n_tokens):
            # print(f'Sentence tokens: {len(word_tokeniser.tokenize(sentence.text))} || Tokens from ZIP {tokens}')
            if token_counter < limit and tokens > 0:
                token_counter += tokens
                # print(f'Adding total tokens {token_counter} Limit {limit} || Appending sentence')
                print(sentence.text)
                chunk.append(sentence.text)
            else:
                # print(f'Token exceeding {limit}, appening sentence to Chunk')
                dic_title.append(title)
                dic_text.append(" ".join(chunk))
                token_counter = 0
                chunk = []
                print(f'Chunck cleared: {chunk}')

    dict = {'filename': dic_title, 'text': dic_text}
    new_df = pd.DataFrame(dict)
    new_df.to_pickle(location)

def create_embeddings(folder):
    openai.api_type = "azure"
    openai.api_base = "https://teamsgptapi.openai.azure.com/"
    openai.api_version = "2023-05-15"
    openai.api_key ="2ae1f9e06b9346748476dcebbdb85a18"
    print("**************************************************")

    infile = os.path.join(folder, 'GPT_Chuncked.pkl')
    outfile = os.path.join(folder, 'chunked_embeddings.pkl')
    
    df_chunk = pd.read_pickle(infile)
    print("###########")
    df_chunk['embeddings'] = df_chunk.apply(lambda a: openai.Embedding.create(input=a['text'], engine='embedding-ada')['data'][0]['embedding'], axis=1)

    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    print('DF for embeddings: ')
    print(df_chunk['embeddings'])
    # df['embeddings'] = df[2][0].apply(eval)

    # embed = df["embeddings"].tolist()
    # print(embed)
    # df.to_csv('testing14.csv')

    print(df_chunk.head())
    df_chunk.to_pickle(outfile)

folder = "E:/PycharmProjects/Notebooks/data"
create_dict(folder)
create_embeddings(folder)